In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation

In [ ]:
train = pd.read_csv("../credit_train.csv", sep = ";", encoding='cp1251')
test = pd.read_csv("../credit_test.csv", sep = ";", encoding='cp1251')

In [ ]:
city_train = pd.read_csv("../datasets/credit_train_clean.csv")
city_test = pd.read_csv("../datasets/credit_test_clean.csv")

In [ ]:
train.head(2)

In [ ]:
y = train['open_account_flg']

In [ ]:
train.info()

In [ ]:
test.info()

I работа с пропусками в данных

In [ ]:
train[train.monthly_income.isnull()]

In [ ]:
train.monthly_income.fillna(value=train.monthly_income.mean(), inplace=True)

In [ ]:
train.living_region = city_train.living_region
test.living_region = city_test.living_region

In [ ]:
train.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
train.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [ ]:
test.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
test.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [ ]:
train.gender = train.gender.map({"M":1, "F":0})
test.gender = test.gender.map({"M":1, "F":0})

In [ ]:
train.rename(columns={"open_account_flg":"target"}, inplace=True)

кодировка категориальных признаков через среднее значения таргета

In [ ]:
train.columns

In [ ]:
job_position_dict = pd.DataFrame(train.groupby("job_position").target.mean()).to_dict()
train['job_position'] = train.job_position.map(job_position_dict["target"])
test['job_position'] = test.job_position.map(job_position_dict["target"])

marital_status_dict = pd.DataFrame(train.groupby("marital_status").target.mean()).to_dict()
train['marital_status'] = train.marital_status.map(marital_status_dict["target"])
test['marital_status'] = test.marital_status.map(marital_status_dict["target"])

edu_dict = pd.DataFrame(train.groupby("education").target.mean()).to_dict()
train['education'] = train.education.map(edu_dict["target"])
test['education'] = test.education.map(edu_dict["target"])

tariff_dict = pd.DataFrame(train.groupby("tariff_id").target.mean()).to_dict()
train['tariff_id'] = train.tariff_id.map(tariff_dict["target"])
test['tariff_id'] = test.tariff_id.map(tariff_dict["target"])

In [ ]:
train_col = train.columns.drop("target")
dummy_df = pd.concat([train[train_col], test], axis=0)
dummy_df = pd.get_dummies(dummy_df, columns=["living_region"], drop_first=True)
train = dummy_df[:train.shape[0]]
test = dummy_df[train.shape[0]:]

In [ ]:
train.monthly_income = train['monthly_income'].astype(float)
test.monthly_income = test['monthly_income'].astype(float)

Генерация данных о зарплате и выплатам по кредиту

In [ ]:
tmp = train['credit_sum'].apply(lambda x: x.replace(",","."))
train['credit_sum'] = tmp
tmp = test['credit_sum'].apply(lambda x: x.replace(",","."))
test['credit_sum'] = tmp

train['credit_sum'] = train['credit_sum'].astype(float)
test['credit_sum'] = test['credit_sum'].astype(float)

credit_pay - ежемесячный расход на кредит

money_for_life - разность между зп и платой по кредиту

In [ ]:
train["credit_pay"] = train["credit_sum"] / train["credit_month"]
train["money_for_life"] = train["monthly_income"] - train["credit_pay"]

test["credit_pay"] = test["credit_sum"] / test["credit_month"]
test["money_for_life"] = test["monthly_income"] - test["credit_pay"]

анализ score_shk

In [ ]:
tmp = train['score_shk'].apply(lambda x: x.replace(",","."))
train.score_shk = tmp

tmp = test['score_shk'].apply(lambda x: x.replace(",","."))
test.score_shk = tmp

train['score_shk'] = train['score_shk'].astype(float)
test['score_shk'] = test['score_shk'].astype(float)

In [ ]:
train['target'] = y

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.to_csv("../datasets/train_target_encode.csv", columns=train.columns, index=False)
test.to_csv("../datasets/test_target_encode.csv", columns=test.columns, index=False)